# Introducion and Business Problem

In todays dyanmic envrionment, so many people keep relocating from one to other place depending on multiple circumstances e.g. relocating due the jobs requirement, better career propect etc. But before reloacting to any new place they need to decide on what are their expectation from the new place then get the relevant information for the that new location and make an informed decision.

The problem I am trying to solve here is if I have to relocate to New York city, and given the option that I can choose any location from the city, which one shall I go for. My requirement for deciding on the place would be 'to have atleast one venue of all the folloing categories in at-max 1 km distance of its center'.
- Play ground / Park / Garden / Field
- School
- Pharmacy stores
- Super Market

# Data

To make the informed decision, first thing would be to find out all the borough in New York city. This I will get from wikipedia page
    https://en.wikipedia.org/wiki/Boroughs_of_New_York_City
I will use the FourSquare APIs, to explore each of the borough to find out which one is the most appropriate for me.

# Methodology

For this exercise, we will follow the below mentioned process.
- Get the list of boroughs from wikipedia link mentioned in data section.
- Get the geographical coordinates of each of the borough using geopy package.
- List all the venues within the range of 1 Km for each borough using FourSquare API.
- Clean the data we have got, to have only the once in which we are interested in as per the requirement.
- Present the data to highliht the findings which will be used for drawing the conclusion of the study.

Import all the packeges and libraires which would be used during the process of this exercise.

In [2]:
import json
import requests
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import pandas as pd
from lxml import etree
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

##### Get the borough information table from the wikipedia

In [3]:
wikiData = requests.get('https://en.wikipedia.org/wiki/Boroughs_of_New_York_City')
html = etree.HTML(wikiData.text)
boroughTable = html.xpath('//table[@class="wikitable sortable"]/tbody/tr')

#### Create a dataframe which will be used to keep the extracted information of each borough of New York city from wikipedia

In [4]:
newyork_df_columns = ['Borough','Latitude','Longitude', 'Population', 'Density(Persons/Sq. Km)']
newyork_borough_df = pd.DataFrame(columns = newyork_df_columns)


#### Create a function which will populate the latitude and longitude details of each borough using geopy library

In [5]:
geolocator = Nominatim(user_agent="ny_explorer")
for row in boroughTable[3:8]:
    rowElements = row.xpath('td')
    name = rowElements[0].xpath('div/b/a')[0].text.strip()

    population_str = rowElements[2].text.strip().split(',')
    population = int("".join(population_str))

    density_str = rowElements[8].text.strip().split(',')
    density = int("".join(density_str))

    location = geolocator.geocode(name)
    latitude = location.latitude
    longitude = location.longitude
    newyork_borough_df = newyork_borough_df.append({'Borough' : name,
                                                    'Latitude' : latitude,
                                                    'Longitude' : longitude,
                                                    'Population' : population,
                                                    'Density(Persons/Sq. Km)' : density}, ignore_index=True)


#### Sort the dataframe of borough and their relevant information based on decending order of their population density

In [6]:
newyork_borough_df = newyork_borough_df.sort_values(by = 'Density(Persons/Sq. Km)',ascending = False)
newyork_borough_df = newyork_borough_df.reset_index(drop=True)

In [7]:
newyork_borough_df

,Borough,Latitude,Longitude,Population,Density(Persons/Sq. Km)
0,Manhattan,40.790087,-73.959829,1664727,27826
1,Brooklyn,40.650104,-73.949582,2648771,14649
2,The Bronx,40.850485,-73.840404,1471160,13231
3,Queens,40.652493,-73.791421,2358582,8354
4,Staten Island,40.583456,-74.149605,479458,3132


#### Get the geographical coordinates of New York City

In [8]:
location = geolocator.geocode('New York City')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Create a function to map all the borough of New York city using Folium pacake, to help in visualizing them on map.

In [9]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(newyork_borough_df['Latitude'], newyork_borough_df['Longitude'], newyork_borough_df['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)


In [10]:
map_newyork

#### Set up the Foursquare credentail which will used while making the request to thier API for getting any information 

In [ ]:
CLIENT_ID = 'Your ID'# your Foursquare ID
CLIENT_SECRET = 'Password for that ID' # your Foursquare Secret
VERSION = '20190820' # Foursquare API version

radius = 1000 # Radius of 1000 meters which is my requirement as mentioned in introduction section
LIMIT = 100 # Limit the number of venues from foursquare requests

#### Define the function to be used to getting the nearby venues of each borough

In [ ]:
def getNearbyVeues(lat, long):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        long, 
        radius, 
        LIMIT)
    return requests.get(url).json()

#### Define the function to get the category of each venue returned by the foursquare API

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### Now get the list of all the nearby venues for each borough and keep them in a dataframe

In [34]:
# This dataframe is created the keep all the venues extracted for all the five borough
nearby_venues_to_all_borrow = pd.DataFrame()

for borough, lat,long in zip(newyork_borough_df['Borough'],newyork_borough_df['Latitude'], newyork_borough_df['Longitude']):
    results = getNearbyVeues(lat, long)

    nearby_venues = json_normalize(results['response']['groups'][0]['items']) # flatten JSON

    # filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues = nearby_venues.loc[:, filtered_columns]

    # filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    nearby_venues['Borough'] = borough
    
    nearby_venues_to_all_borrow = nearby_venues_to_all_borrow.append(nearby_venues)
 

##### See a total of how many venues have we got for all the borough

In [37]:
nearby_venues_to_all_borrow.shape

(349, 5)

##### Clean the dataframe of all the venues to have the ordered indexes, which are in an unordered fashion due to venues being appended one after the other for each borough, and each append call creates a new index starting from 0

In [38]:
nearby_venues_to_all_borrow =  nearby_venues_to_all_borrow.reset_index(drop=True)

##### Let's print the dataframe to see what it has

In [41]:
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 500)
nearby_venues_to_all_borrow

,name,categories,lat,lng,Borough
0,North Meadow,Park,40.792027,-73.959853,Manhattan
1,Central Park Tennis Center,Tennis Court,40.789313,-73.961862,Manhattan
2,East Meadow,Field,40.790160,-73.955498,Manhattan
3,Oldest Tree in Central Park,Park,40.789188,-73.957867,Manhattan
4,Central Park - North Meadow Recreation Center,Playground,40.790939,-73.960304,Manhattan
5,Central Park - Gate Of All Saints,Park,40.791591,-73.964795,Manhattan
6,The Jewish Museum,Museum,40.785276,-73.957411,Manhattan
7,Da Capo,Café,40.787679,-73.953899,Manhattan
8,K&D Wines & Spirits,Wine Shop,40.787096,-73.954261,Manhattan
9,Russ & Daughters,Kosher Restaurant,40.785332,-73.957481,Manhattan


##### Define the features sets which are part of my requirement as mentioned in introduction section and then keep only those entries in dataframe which are part of my requirement

In [83]:
features = ['Park','School','Playground','Field','Tennis Court','Garden','Museum','Pharmacy','Supermarket','Medical']
borough_venues  = nearby_venues_to_all_borrow
borough_venues = borough_venues[borough_venues['categories'].isin(features)]

#### Let's see it again as to how many entries are in dataframe after the non-mandatory venues has bee filtered out

borough_venues.shape

##### Reset the index of dataframe which has been disturbed due to some rows being droped from the previous operations

In [84]:
borough_venues = borough_venues.reset_index(drop=True)

##### Re-order the columns of dataframe to make it more readable while having the quick view

In [85]:
borough_venues = borough_venues.reindex(columns = ['Borough','categories','name','lat','lng'])
borough_venues

,Borough,categories,name,lat,lng
0,Manhattan,Park,North Meadow,40.792027,-73.959853
1,Manhattan,Tennis Court,Central Park Tennis Center,40.789313,-73.961862
2,Manhattan,Field,East Meadow,40.790160,-73.955498
3,Manhattan,Park,Oldest Tree in Central Park,40.789188,-73.957867
4,Manhattan,Playground,Central Park - North Meadow Recreation Center,40.790939,-73.960304
5,Manhattan,Park,Central Park - Gate Of All Saints,40.791591,-73.964795
6,Manhattan,Museum,The Jewish Museum,40.785276,-73.957411
7,Manhattan,Park,Central Park - Woodman's Gate,40.787786,-73.955924
8,Manhattan,Park,Central Park - 99th & 5th Ave,40.788791,-73.955232
9,Manhattan,Playground,Central Park - Wild West Playground,40.789715,-73.965471


##### Drop the columns which are no more required for further analysis

In [ ]:
borough_venues.drop(['name','lat','lng'], axis=1, inplace=True)
borough_venues

# Results

##### Now when we are done with all the steps we intended to carry out, lets see how many venues do we have for each category in each borough.
Group earch of the borough for each category of venues (the one which are part of the original requirement mentione in introduction section)

In [88]:
borough_venues.groupby(['Borough','categories'])['Borough'].describe()



count unique            top freq
Borough       categories                                   
Brooklyn      Pharmacy         5      1       Brooklyn    5
              Playground       1      1       Brooklyn    1
              School           1      1       Brooklyn    1
              Supermarket      4      1       Brooklyn    4
Manhattan     Field            1      1      Manhattan    1
              Garden           3      1      Manhattan    3
              Museum           1      1      Manhattan    1
              Park            11      1      Manhattan   11
              Playground       6      1      Manhattan    6
              Tennis Court     1      1      Manhattan    1
Staten Island Park             1      1  Staten Island    1
              Pharmacy         1      1  Staten Island    1
The Bronx     Park             1      1      The Bronx    1
              Pharmacy         2      1      The Bronx    2
              Supermarket      3      1      The Bronx    3

The count column of the dataframe shows the number of venues that borough has for that category.

# Discussion

The initial thought which I had before staring this exercise was that the borough 'Manhattan' which is the most corwded borough out of the five
New York city is most likely to meet my requirement. Though it has more number of overall venues(23) and which is more than combined total of 
all other four boroughs (19), it doesn't meet my requirement and misses on not having a school within 1 Km distance from its center. This study
also highlights that sometimes any specific area may not have all the facilities which anyone can expect to, though they have more number overall
facilities offered by any other areas.

Though I have limite the number of venues to be 100 while getting all the nearby venues for each borough, increasing this limit doens't yeild any better results and none of the borough has more than 100 venues in range of 1 Km.

# Conclusion

After analysing data and matching them against requirements, it can be concluded that the only borough which meets the requirement is 'Brooklyn' and that should be the place to which I can look forward to shift to.